# Ex16


Um navio tem três compartimentos de carga: **proa**, **centro** e **popa**. As capacidades limites são:

| Compartimento | Peso (ton) | Volume (m³) |
|--------------|------------|-------------|
| **Proa**     | 2.000      | 30.000      |
| **Centro**   | 3.000      | 40.000      |
| **Popa**     | 1.500      | 20.000      |

A empresa de navegação, proprietária do navio, pode aceitar toda ou parte das seguintes cargas:

| Carga | Quantidade (ton) | Volume Específico (m³/ton) | Lucro (R$/ton) |
|-------|----------------|--------------------|----------------|
| **A** | 6.000         | 60                 | 6.000         |
| **B** | 4.000         | 50                 | 8.000         |
| **C** | 2.000         | 25                 | 5.000         |

Para preservar o equilíbrio do navio, o peso em cada compartimento deve ser proporcional à sua capacidade em toneladas.

Formule um modelo matemático de programação linear para determinar como carregar o navio de modo a **maximizar o lucro**, respeitando as restrições de capacidade e equilíbrio da carga.



## **Definição das Variáveis de Decisão**
Seja $x_{ij}$ a quantidade (em toneladas) da carga $i$ alocada ao compartimento $j$, onde:
- $i \in \{A, B, C\}$ representa os tipos de carga;
- $j \in \{\text{Proa}, \text{Centro}, \text{Popa}\}$ representa os compartimentos de carga do navio.



## **Função Objetivo**
Deseja-se **maximizar** o lucro total obtido pelo transporte das cargas:
$$\max Z = \sum_{i \in \{A,B,C\}} \sum_{j \in \{\text{Proa},\text{Centro},\text{Popa}\}} p_i x_{ij}$$
Onde:
- $p_i$ representa o lucro por tonelada da carga $i$;
- $x_{ij}$ é a quantidade da carga $i$ no compartimento $j$.



## **1 Restrições de Capacidade de Peso**
O peso alocado em cada compartimento não pode exceder sua capacidade:
$$\sum_{i \in \{A,B,C\}} x_{i,\text{Proa}} \leq 2000$$
$$\sum_{i \in \{A,B,C\}} x_{i,\text{Centro}} \leq 3000$$
$$\sum_{i \in \{A,B,C\}} x_{i,\text{Popa}} \leq 1500$$



## **2 Restrições de Capacidade de Volume**
O volume total ocupado pelas cargas em cada compartimento não pode exceder sua capacidade volumétrica:
$$\sum_{i \in \{A,B,C\}} v_i x_{i,\text{Proa}} \leq 30000$$
$$\sum_{i \in \{A,B,C\}} v_i x_{i,\text{Centro}} \leq 40000$$
$$\sum_{i \in \{A,B,C\}} v_i x_{i,\text{Popa}} \leq 20000$$
Onde $v_i$ representa o volume específico da carga $i$ (m³/ton).



## **3 Restrições de Disponibilidade de Carga**
A quantidade total de cada tipo de carga alocada não pode ultrapassar sua disponibilidade máxima:
$$\sum_{j \in \{\text{Proa},\text{Centro},\text{Popa}\}} x_{A,j} \leq 6000$$
$$\sum_{j \in \{\text{Proa},\text{Centro},\text{Popa}\}} x_{B,j} \leq 4000$$
$$\sum_{j \in \{\text{Proa},\text{Centro},\text{Popa}\}} x_{C,j} \leq 2000$$


## **4 Restrições de Equilíbrio do Peso**
Para preservar o equilíbrio do navio, o peso total em cada compartimento deve ser proporcional à sua capacidade em toneladas:
$$\frac{\sum_{i \in \{A,B,C\}} x_{i,\text{Proa}}}{2000} = \frac{\sum_{i \in \{A,B,C\}} x_{i,\text{Centro}}}{3000} = \frac{\sum_{i \in \{A,B,C\}} x_{i,\text{Popa}}}{1500}$$



## **5 Restrição de Não-Negatividade**
O modelo considera que não pode haver quantidades negativas de carga:
$$x_{ij} \geq 0, \quad \forall i \in \{A, B, C\}, \quad \forall j \in \{\text{Proa}, \text{Centro}, \text{Popa}\}$$




In [17]:
import pyomo.environ as pyo

In [18]:
# Criação do modelo
model = pyo.ConcreteModel()

In [19]:

# Conjuntos
model.Cargas = pyo.Set(initialize=["A", "B", "C"])

# Compartimentos do navio
model.Compartimentos = pyo.Set(initialize=["Proa", "Centro", "Popa"])

In [20]:
# Lucro por tonelada de carga
lucro = {"A": 6000, "B": 8000, "C": 5000}
model.p_lucro = pyo.Param(model.Cargas, initialize=lucro)

# Capacidade máxima de peso em cada compartimento
capacidade_peso = {"Proa": 2000, "Centro": 3000, "Popa": 1500}
model.cap_peso = pyo.Param(model.Compartimentos, initialize=capacidade_peso)

# Capacidade máxima de volume em cada compartimento
capacidade_volume = {"Proa": 30000, "Centro": 40000, "Popa": 20000}
model.cap_volume = pyo.Param(model.Compartimentos, initialize=capacidade_volume)

# Volume específico por tonelada de carga
volume_especifico = {"A": 60, "B": 50, "C": 25}
model.v_especifico = pyo.Param(model.Cargas, initialize=volume_especifico)

# Disponibilidade máxima de carga
disponibilidade = {"A": 6000, "B": 4000, "C": 2000}
model.disp_carga = pyo.Param(model.Cargas, initialize=disponibilidade)

In [21]:

# ------------------------------
# Variáveis de Decisão
# ------------------------------

model.x = pyo.Var(model.Cargas, model.Compartimentos, domain=pyo.NonNegativeReals)


In [22]:

# ------------------------------
# Função Objetivo
# ------------------------------
def fo(model):
    return sum(model.x[i, j] * model.p_lucro[i] for i in model.Cargas for j in model.Compartimentos)

model.objetivo = pyo.Objective(rule=fo, sense=pyo.maximize)


In [ ]:
# ------------------------------
# Restrições
# ------------------------------

# 1. Restrição de capacidade de peso por compartimento
def capacidade_peso_rule(model, j):
    return sum(model.x[i, j] for i in model.Cargas) <= model.cap_peso[j]

model.capacidade_peso = pyo.Constraint(model.Compartimentos, rule=capacidade_peso_rule)

# 2. Restrição de capacidade de volume por compartimento
def capacidade_volume_rule(model, j):
    return sum(model.x[i, j] * model.v_especifico[i] for i in model.Cargas) <= model.cap_volume[j]

model.capacidade_volume = pyo.Constraint(model.Compartimentos, rule=capacidade_volume_rule)

# 3. Restrição de disponibilidade de carga
def disponibilidade_rule(model, i):
    return sum(model.x[i, j] for j in model.Compartimentos) <= model.disp_carga[i]

model.disponibilidade = pyo.Constraint(model.Cargas, rule=disponibilidade_rule)

# 4. Restrição de equilíbrio de peso entre os compartimentos
def equilibrio_peso_rule_1(model):
    peso_proporcao = sum(model.x[i, "Proa"] for i in model.Cargas) / model.cap_peso["Proa"]
    peso_centro = sum(model.x[i, "Centro"] for i in model.Cargas) / model.cap_peso["Centro"]
    return peso_proporcao == peso_centro #== peso_popa
def equilibrio_peso_rule_2(model):
    peso_centro = sum(model.x[i, "Centro"] for i in model.Cargas) / model.cap_peso["Centro"]
    peso_popa = sum(model.x[i, "Popa"] for i in model.Cargas) / model.cap_peso["Popa"]
    return peso_centro == peso_popa
def equilibrio_peso_rule_3(model):
    peso_proporcao = sum(model.x[i, "Proa"] for i in model.Cargas) / model.cap_peso["Proa"]
    peso_popa = sum(model.x[i, "Popa"] for i in model.Cargas) / model.cap_peso["Popa"]
    return  peso_popa == peso_proporcao
model.equilibrio_peso = pyo.Constraint(rule=equilibrio_peso_rule_1)
model.equilibrio_peso_2 = pyo.Constraint(rule=equilibrio_peso_rule_2)
model.equilibrio_peso_3 = pyo.Constraint(rule=equilibrio_peso_rule_3)



In [24]:
# ------------------------------
# Escrita do Modelo em Arquivo
# ------------------------------
model.write("ex16.lp", io_options={"symbolic_solver_labels": True})

('ex16.lp', 1245907519472)

In [25]:
# ------------------------------
# Resolução
# ------------------------------
solver = pyo.SolverFactory("appsi_highs")
results = solver.solve(model, tee=True)

# Exibindo resultados
print("\nStatus do solver:", results.solver.status)
print("Condição de terminação:", results.solver.termination_condition)
valor_obj = pyo.value(model.objetivo)
formatted = f"{valor_obj:,.2f}"
formatted = formatted.replace(",", "X").replace(".", ",").replace("X", ".")
print("Valor da Função Objetivo: R$", formatted)

LP   has 12 rows; 9 cols; 45 nonzeros
Coefficient ranges:
  Matrix [3e-04, 6e+01]
  Cost   [5e+03, 8e+03]
  Bound  [0e+00, 0e+00]
  RHS    [2e+03, 4e+04]
Presolving model
10 rows, 9 cols, 39 nonzeros  0s
Dependent equations search running on 3 equations with time limit of 1000.00s
Dependent equations search removed 1 rows and 6 nonzeros in 0.00s (limit = 1000.00s)
9 rows, 9 cols, 33 nonzeros  0s
Presolve : Reductions: rows 9(-3); columns 9(-0); elements 33(-12)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -4.1599492483e+05 Ph1: 8(22.9519); Du: 9(415995) 0s
          8    -1.6400000000e+07 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model status        : Optimal
Simplex   iterations: 8
Objective value     :  1.6400000000e+07
Relative P-D gap    :  0.0000000000e+00
HiGHS run time      :          0.00

Status do solver: ok
Condição de terminação: optimal
Valor da Função Ob

In [26]:
print("Resultados do modelo:")
for i in model.Cargas:
    for j in model.Compartimentos:
        valor = pyo.value(model.x[i, j])
        print(f"Carga {i} no compartimento {j}: {valor:.2f} toneladas")



Resultados do modelo:
Carga A no compartimento Proa: 0.00 toneladas
Carga A no compartimento Centro: 0.00 toneladas
Carga A no compartimento Popa: 0.00 toneladas
Carga B no compartimento Proa: 338.46 toneladas
Carga B no compartimento Centro: 307.69 toneladas
Carga B no compartimento Popa: 153.85 toneladas
Carga C no compartimento Proa: 523.08 toneladas
Carga C no compartimento Centro: 984.62 toneladas
Carga C no compartimento Popa: 492.31 toneladas
